In [1]:
import pandas as pd
import numpy as np
import re

def load_dataframe(relative_path,dataframe_name):
    df = pd.read_pickle(f'{relative_path}/{dataframe_name}.pkl')    
    return df

def read_file(relative_path,file_name):
    text= ""
    with open(f'{relative_path}/{file_name}.abc','r') as f:
        text = f.read()
    return text

In [2]:
relative_path ="notebooks/data/final_dataset"
filename_name = 'clean_augmented_data'
#filename_name = 'clean_original_training_data'
#relative_path ="notebooks/data/original_dataset"
training_data_df = load_dataframe(relative_path,filename_name)
training_data_df.columns


Index(['unit_note_length', 'tuneBook', 'title', 'reference_number',
       'original_header', 'original_body', 'meter', 'key', 'clean_song',
       'clean_header', 'clean_body', 'chord_progression', '"fm"', '"ff'"',
       '"f7"', '"em"', '"ee'"', '"e7"', '"e"', '"dm"', '"dd'"', '"d7"', '"d"',
       '"cm"', '"cc'"', '"c7"', '"c#m"', '"c#7"', '"c"', '"Gm"', '"Gg"',
       '"Gd'"', '"G7"', '"G#m"', '"G#7"', '"G"', '"Fm"', '"Ff"', '"Fc'"',
       '"F7"', '"F#m"', '"F#7"', '"F"', '"Em"', '"Eb"', '"E7"', '"E#m"',
       '"E#7"', '"E"', '"Dm"', '"Da"', '"D7"', '"D#m"', '"D#7"', '"D"', '"Cm"',
       '"Cg"', '"C7"', '"C#m"', '"C#7"', '"C"', '"Bm"', '"Bf"', '"Bb"', '"B7"',
       '"B#m"', '"B#7"', '"B"', '"Am"', '"Ae'"', '"Aa"', '"A7"', '"A#m"',
       '"A#7"', '"A"'],
      dtype='object')

In [3]:
training_data_df.tail()

,unit_note_length,tuneBook,title,reference_number,original_header,original_body,meter,key,clean_song,clean_header,...,"""B#m""","""B#7""","""B""","""Am""","""Ae'""","""Aa""","""A7""","""A#m""","""A#7""","""A"""
9491,1/4,dataset_min5.abc,Grandpa's,78,X:78\nT:Grandpa's\nM:4/4\nL:1/4\nK:Amajor,"E/2D/2|""A,""CE""E7""FG|""A,""A/2G/2A/2B/2ce|""B,m""dc...",4/4,A,"M:4/4\nL:1/4\nK:A\n|""A""""E7""|""A""|""Bm""""B7""|""E7""|...","M:4/4\nL:1/4\nK:A\n|""A""""E7""|""A""|""Bm""""B7""|""E7""|...",...,0,0,0,0,0,0,0,0,0,9
9492,1/4,dataset_min5.abc,The Girl With The Green Hat On,79,X:79\nT:The Girl With The Green Hat On\nM:4/4\...,"(3E/2F/2G/2|""A,""AE""E7""E/2F/2E/2D/2|""A,""C/2D/2E...",4/4,A,"M:4/4\nL:1/4\nK:A\n|""A""""E7""|""A""""E7""|""A""|""E7""|""...","M:4/4\nL:1/4\nK:A\n|""A""""E7""|""A""""E7""|""A""|""E7""|""...",...,0,0,0,0,0,0,0,0,0,9
9493,1/4,dataset_min5.abc,Green Meadow,80,X:80\nT:Green Meadow\nM:4/4\nL:1/4\nK:Dmajor,"(3A,/2B,/2C/2|""D""DD/2E/2F/2D/2F/2A/2|""G,""B/2c/...",4/4,D,"M:4/4\nL:1/4\nK:D\n|""D""|""G""""D""|""Em""|""Em""""A7""|""...","M:4/4\nL:1/4\nK:D\n|""D""|""G""""D""|""Em""|""Em""""A7""|""...",...,0,0,0,0,0,0,7,0,0,0
9494,1/4,dataset_min5.abc,The Old Grey Cat,82,X:82\nT:The Old Grey Cat\nM:4/4\nL:1/4\nK:Bminor,"F|""B,m""BBB,B,/2C/2|""B,m""D/2C/2D/2E/2F/2E/2F/2^...",4/4,Bm,"M:4/4\nL:1/4\nK:Bm\n|""Bm""|""Bm""|""A""|""A""|""Bm""|""B...","M:4/4\nL:1/4\nK:Bm\n|""Bm""|""Bm""|""A""|""A""|""Bm""|""B...",...,0,0,0,0,0,0,0,0,0,5
9495,1/4,dataset_min5.abc,Gyre And Gimble,84,X:84\nT:Gyre And Gimble\nM:4/4\nL:1/4\nK:Amajor,"E|""A,""AECE|""B,m""FD""E7""B,D|""A,""CEA3/2B/2|""E7""c/...",4/4,A,"M:4/4\nL:1/4\nK:A\n|""A""|""Bm""""E7""|""A""|""E7""""A""|""...","M:4/4\nL:1/4\nK:A\n|""A""|""Bm""""E7""|""A""|""E7""""A""|""...",...,0,0,0,0,0,0,0,0,0,12


In [4]:
training_data_df["clean_header"].str.len().max()

373

In [5]:
training_data_df["clean_body"].str.len().max()

1257

In [6]:
bodies = ""
silences = 0
for body in training_data_df["clean_body"]:
    if 'z' in body:
        silences +=1 
    bodies += body+"\n"
chars = sorted(list(set(bodies)))
vocab_size = len(chars)
print('vocab: ',''.join(chars))
print('vocab_size',vocab_size)
print("silences ",silences)

vocab:  
"#'(),-/1234567=ABCDEFG[]^_abcdefgmz|~
vocab_size 39
silences  516


In [7]:
training_data_text = read_file(relative_path,filename_name)

print("number of chars:",len(training_data_text))

number of chars: 4062773


In [7]:
chars = sorted(list(set(training_data_text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

NameError: name 'training_data_text' is not defined

In [1]:
import torch

print(torch.__version__)

2.0.0+cu117


In [2]:
import transformers
print(transformers.__version__)

4.28.0.dev0


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import wandb
import tiktoken

print(wandb.__version__)

0.14.2


In [1]:
!ls

Dockerfile  docker-compose.yaml  overrides.json
README.md   notebooks		 requirements.txt


In [2]:
from dotenv import load_dotenv
import os
nano_path = 'notebooks/nanoGPT'
os.chdir(nano_path)
load_dotenv()

True

In [3]:
!ls

LICENSE      assets	      data	  out-abc-char	wandb
README.md    config	      model.py	  sample.py
__pycache__  configurator.py  older_ckpt  train.py


In [ ]:
# Dataset with multiple voices present
#length of dataset in characters: 4,149,703
#all the unique characters: 
#"#'()+,-/123456789:=ABCDEFGKLM[]^_abcdefgmz|~
#vocab size: 46
#train has 3,734,732 tokens
#val has 414,971 tokens

In [5]:
!python3 data/abc_char/prepare.py

length of dataset in characters: 4,062,773
all the unique characters: 
"#'(),-/123456789:=ABCDEFGKLM[]^_abcdefgmz|~
vocab size: 45
train has 3,656,495 tokens
val has 406,278 tokens


In [6]:
!python3 train.py config/train_abc_char.py

Overriding config with config/train_abc_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-abc-char'
eval_interval = 10 # keep frequent because we'll overfit
eval_iters = 500
log_interval = 5 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = True # override via command line if you like
wandb_project = 'abc-char'
wandb_run_name = 'mini-char-gpt-hd-8-ly-12-embd-240'

dataset = 'abc_char'
batch_size = 32
block_size = 512 # context of up to 512 previous characters

# baby GPT model :)
n_layer = 12
n_head = 8
n_embd = 240
dropout = 0.2

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 5000
lr_decay_iters = 5000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of tokens per iter is small

warmup_iters = 

## Test key with most occurrences: G

In [7]:
!python3 sample.py --out_dir=out-abc-char --start='M:4/4L:1/4K:G|"G"|"C"|"D"|"D"|"G"|"C"|"D"|"G"|]'

Overriding: out_dir = out-abc-char
Overriding: start = M:4/4L:1/4K:G|"G"|"C"|"D"|"D"|"G"|"C"|"D"|"G"|]
number of parameters: 8.31M
abc_char
Loading meta from data/abc_char/meta.pkl...
M:4/4L:1/4K:G|"G"|"C"|"D"|"D"|"G"|"C"|"D"|"G"|]
D|"G"d/2B/2d/2g/2|"C"e/2G/2A/2c/2|"D"BA|"D"dd/2c/2|"G"B/2A/2G/2F/2|"C"G/2B/2G/2c/2|"D"d/2c/2B/2A/2|"G"G3/2A/2|"C"G/2E/2G/2c/2|"D"B/2A/2B/2A|"G"G2|G3/2B/2|"C"e/2d/2e/2f/2g/2|"D"a/2f/2e/2g/2f/2d/2|"G"Bd/2d/2|"D"B/2f/2e/2g/2f/2d/2|"G"g3/2a/2|"C"g/2e/2g/2f/2g/2|"D"f/2e/2d/2c/2|"G"B/2G/2Ge/2B/2|"G"G3|]

M:4/4
L:1/4
K:C
|"C"|"G7"|"C"|"F"|"G7"|"C"|"C"|"F"|"G7"|"C"|"C"|"F"|"G7"|"C""G7"|"C"|"G"|"Am""D7"|"G7"|"C"|]
c/2B/2|"C"cGAG|"C"cGAG|"F"cAGA|"G7"BgBd|"C"cGAG|"C"cGAG|"F"cAGA|"G7"BgBd|"C"c2"G7"c|"C"c2c
---------------
M:4/4L:1/4K:G|"G"|"C"|"D"|"D"|"G"|"C"|"D"|"G"|]
B/2c/2|"G"d/2B/2B/2B/2B/2B/2B/2c/2|d/2B/2B/2B/2B/2B/2B/2|"C"c/2G/2G/2E/2CB/2A/2|G/2E/2G/2c/2G/2E/2G/2|"D"D/2E/2D/2E/2F/2D/2E/2F/2|"G"d/2B/2B/2B/2B/2B/2B/2B/2|"C"c/2G/2C/2E/2Cc/2G/2|"D"D/2E/2D/2E/2D/2E/2F/

## Test major key with low samples: C

In [10]:
!python3 sample.py --out_dir=out-abc-char --start='M:4/4L:1/4K:C|"C"|"F"|"G"|"G"|"C"|"F"|"G"|"C"|]'

Overriding: out_dir = out-abc-char
Overriding: start = M:4/4L:1/4K:C|"C"|"F"|"G"|"G"|"C"|"F"|"G"|"C"|]
number of parameters: 8.31M
abc_char
Loading meta from data/abc_char/meta.pkl...
M:4/4L:1/4K:C|"C"|"F"|"G"|"G"|"C"|"F"|"G"|"C"|]
GF|"C"E2EE|"F"F2A3/2A/2|"G"BGGA|"C"AGGA|"C"cccd/2e/2|"F"fAGF|"G"GBGA|"C"cc3|"F"cde|"G"dg/2e/2f/2e/2d/2|"C"c2c3/2e/2|"F"fAGF|"G"GBGA|"C"ccc|]

M:4/4
L:1/4
K:C
|"C"|"G7"|"C"|"G7"|"C"|"G7"|"C"|"C"|"C"|"G7"|"C"|"G7"|"C"|"C"|"C"|"F"|"G7"|"C"|]
e/2f/2|"C"gec|"G7"Bd/2e/2d/2c/2|"C"cec|"G7"dBe/2f/2|"C"gec|"G7"Bd/2e/2d/2c/2|"C"c2c|e/2f/2|"C"gecf/2g/2|"C"gecd/2e/2|"G7"gg/2g/2f/2e/2|"C"gecd/2e/2|"G7"gG/2A/2B/2d/2c/2|"C"c2c2|"C"gecf/2g/2|"C"c'ecc/2g/2|"F"c'ecd/2e/2|"G7"fdBd|"C"c2c|]

M:2/2
L:1/8
K:G
|"G"|"G
---------------
M:4/4L:1/4K:C|"C"|"F"|"G"|"G"|"C"|"F"|"G"|"C"|]
G/2|"C"c/2c/2c/4A/4|G/2c/2|"F"F/2d/2c/2|c3/4B/4A/2G/2|"G"G/2A/2B/2|"C"c/2G/2A/2|G3/2|"F"F/2A/2c/2|"G"B3/4A/4G/2G/2|"C"c/2c/4A/2G/2|"F"A/2c/2f/2|e/2d/2c/2|"G"B/2d/2B/2|"C"c/2G/2A/2G/2|"F"F/2A/2c/2|"G"B/2d/

## Test minor key with low samples: Am

In [9]:
!python3 sample.py --out_dir=out-abc-char --start='M:4/4L:1/4K:Am|"Am"|"Dm"|"E"|"E"|"Am"|"Dm"|"E"|"Am"|]'

Overriding: out_dir = out-abc-char
Overriding: start = M:4/4L:1/4K:Am|"Am"|"Dm"|"E"|"E"|"Am"|"Dm"|"E"|"Am"|]
number of parameters: 8.31M
abc_char
Loading meta from data/abc_char/meta.pkl...
M:4/4L:1/4K:Am|"Am"|"Dm"|"E"|"E"|"Am"|"Dm"|"E"|"Am"|]
A|"Am"A/2B/2A/2B/2A/2B/2|c/2B/2A/2B/2A/2B/2c/2|"Dm"d/2e/2d/2e/2d/2B/2A/2|"E"^G/2A/2B/2G/2Ee/2d/2|"Am"e/2e/2A/2B/2A/2B/2c/2d/2|"G"e/2d/2B/2G/2E/2G/2A/2B/2|"Am"c/2B/2A/2"Dm"F/2A/2A|"E"G/2B/2c/2B/2E/2G/2B/2|"Am"c/2A/2B/2G/2A/2B/2c/2B/2|"Dm"AF/2A/2"Dm"A|]

M:4/4
L:1/4
K:C
|"C"|"C"|"Dm""G7"|"C"|"C"|"Dm""G7"|"C"|"C"|"C""G7"|"C"|"C""G7"|"C"|"F""G7"|"C""G7"|"C"|"C"|"G7"|"C"|"C"|"Dm""G7"|"C"|]
D|"C"CC/2C/2C/2E/2G/2c/2|"C"c/2G/2c/2d/2e/2g/2f/2e/2|"Dm"d/2c/2B/2A/2"G7"GD/2D/2|"C"CCC/2C/2E/2G/2c/2|"C"c/2G/2
---------------
M:4/4L:1/4K:Am|"Am"|"Dm"|"E"|"E"|"Am"|"Dm"|"E"|"Am"|]
"Am"eAA/2B/2|"Dm"AA/2B/2A/2B/2|"E"G/2F/2E/2D/2E/2F/2|"Am"GEEC|"Dm"D2E3/2F/2|"E"G/2F/2E/2D/2E/2F/2|"E"E3/2C/2B,/2A,/2G,/2|"Am"A,3/2C/2B,B,/2|"Dm"A,3/2F/2F/2E/2D/2|"E"^G3/2E/2G/2F/2E/2|"Am

## Test older checkpoint

In [24]:
 !python3 sample.py --out_dir=older_ckpt/m_voices --path_meta=older_ckpt/m_voices --start='M:4/4L:1/4K:G|"G"|"C"|"D"|"D"|"G"|"C"|"D"|"G"|]'

Overriding: out_dir = older_ckpt/m_voices
Overriding: path_meta = older_ckpt/m_voices
Overriding: start = M:4/4L:1/4K:G|"G"|"C"|"D"|"D"|"G"|"C"|"D"|"G"|]
number of parameters: 14.18M
shakespeare_char
Loading meta from older_ckpt/m_voices/meta.pkl...
M:4/4L:1/4K:G|"G"|"C"|"D"|"D"|"G"|"C"|"D"|"G"|]
e/2|"G"gg/2a/2gd/2c/2|"G"B/2G/2A/2G/2[+GB][+GB][+GB]|"D"A/2G/2F/2E/2D/2F/2A/2c/2|"G"B/2[GB][G/2B/2][+GB]e/2f/2|"G"gg/2a/2g/2e/2d/2e/2|g/2f/2g/2a/2bb/2e/2|"D"f/2e/2f/2g/2a/2f/2a/2f/2|"G"g/2f/2g/2a/2"C"g/2e/2d/2e/2|"G"gg/2a/2g/2e/2d/2e/2|g/2f/2g/2a/2bb|"D"d'd'"D7"e/2f3/2|"G"g3/2a/2gG/2E/2|"G"D/2B,/2D/2G/2D/2B,/2D/2G/2|"C"E/2[CE][C/2E/2][CE][CE]|"D"A/2B/2A/2F/2D/2F/2A/2c/2|"G"B/2[GB][G/2B/2][GB]G/2E/2|"G"D/2B,/2D/2G/2D/2B,/2D/2G/2|"C"E/2[CE][C/2E/2][CE][CE]|"D"A/2B/2A/2F/2D/2F/2A/2c/2|"G"B/2[GB][G
---------------
M:4/4L:1/4K:G|"G"|"C"|"D"|"D"|"G"|"C"|"D"|"G"|]
e/2|"G"gg/2a/2gd/2c/2|"G"B/2G/2A/2G/2[+GB][+GB]|"D"A/2G/2F/2E/2D/2F/2A/2c/2|"G"B/2[GB][G/2B/2][+GB]e/2f/2|"G"gg/2a/2g/2e/2d/2c/2|"G"B/2G/2

In [ ]:
cat older_ckpt/m_voices/ckpt.pkl

In [ ]:
!ls -l older_ckpt/m_voices

In [ ]:
!ls -l out-abc-char/

In [ ]:
!id